In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa

Создаю фрейм для наших данных

In [2]:
train = pd.read_csv('../new-data-without_silence/train_gt.csv', header=None, names=['name', 'label'])

In [3]:
audio_path = "../new-data-without_silence/train/ffbd950f-341c-4798-a5cf-2e465c59c48b.mp3"

Загружаем файл

In [4]:
sample_rate = 22050 # указываем желаемую частоту дискретизации
FRAME_SIZE = 256 # Размер одного фрейма, так сказать кадра (аудио мы делим на кадры, а в кадрах есть семплы(малые кусочки))
HOP_LENGTH = 64 # Максимальный шаг, на который перебираемся, то есть это шаг наш, а фрейм-берущийся небольшой объем данных
FRAME_SIZE_FFT = 512 # размер фрейма для преобразования фурье, там лучше побольше делать 
HOP_SIZE_FFT = 128 # соответственно, шаг для фурье
audio, sr = librosa.load(audio_path, sr=sample_rate) # загрузка

Функция амплитудной огибающей, дает поверхностный рисунок (типо волна)

In [5]:
def amplitude_envelope(signal, frame_size, hop_length):
    return np.array([max(signal[i:i+frame_size]) for i in range(0, signal.size, hop_length)])

Здесь амплитудная вычисляется, а также средне квадратичное значение каждого фрейма и скорость пересечения нуля (насколько часто меняется знак)

In [6]:
ae_audio = amplitude_envelope(audio, FRAME_SIZE, HOP_LENGTH)
rms_audio = librosa.feature.rms(y=audio, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
zcr_audio = librosa.feature.zero_crossing_rate(y=audio, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
print(ae_audio.shape)
print(rms_audio.shape)
print(zcr_audio.shape)

(2199,)
(2199,)
(2199,)


Здесь идет кратковременное преобразование Фурье

In [7]:
S_scale = librosa.stft(audio, n_fft=FRAME_SIZE_FFT, hop_length=HOP_SIZE_FFT)
stft_audio = np.abs(S_scale)

Мел-кепcтральные коэффициенты (MFCC) -- очень важная вещь

In [8]:
mfccs = librosa.feature.mfcc(y=audio, n_mfcc=13, sr=sample_rate)
delta_mfccs = librosa.feature.delta(mfccs)
delta2_mfcc = librosa.feature.delta(mfccs, order=2)
mfccs_audio = np.concatenate((mfccs, delta_mfccs, delta2_mfcc))

Спектральный центроид

In [9]:
cent_audio = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)
print(cent_audio.shape)

(1, 275)


Спектральный спад

In [10]:
down_audio = librosa.feature.spectral_rolloff(y=audio, sr=sample_rate)
print(down_audio.shape)

(1, 275)


Теперь группируем

Я создал списки по среднеквадратичному и отклонению

In [11]:
class Transformer():

    def __init__(self) -> None:
        self.sample_rate = 22050 
        self.FRAME_SIZE = 256 
        self.HOP_LENGTH = 64 
        self.FRAME_SIZE_FFT = 512 
        self.HOP_SIZE_FFT = 128

    def amplitude_envelope(self, audio, frame_size, hop_length):
        return np.array([max(audio[i:i+frame_size]) for i in range(0, audio.size, hop_length)])

    def mfccs_ob(self, audio, sample_rate):
        mfccs = librosa.feature.mfcc(y=audio, n_mfcc=13, sr=sample_rate)
        delta_mfccs = librosa.feature.delta(mfccs)
        delta2_mfcc = librosa.feature.delta(mfccs, order=2)
        mfccs_audio = np.concatenate((mfccs, delta_mfccs, delta2_mfcc))
        return mfccs_audio

    def obrabotka(self, audio_path):
        audio, sr = librosa.load(audio_path, sr=self.sample_rate)
        amplitude_list.append(self.amplitude_envelope(audio, self.FRAME_SIZE, self.HOP_LENGTH))
        mfccs_list.append(self.mfccs_ob(audio, self.sample_rate))
        rms_list.append(librosa.feature.rms(y=audio, frame_length=self.FRAME_SIZE, hop_length=self.HOP_LENGTH)[0])
        zcr_list.append(librosa.feature.zero_crossing_rate(y=audio, frame_length=self.FRAME_SIZE, hop_length=self.HOP_LENGTH)[0])
        stft_list.append(np.abs(librosa.stft(audio, n_fft=self.FRAME_SIZE_FFT, hop_length=self.HOP_SIZE_FFT)))
        cent_list.append(librosa.feature.spectral_centroid(y=audio, sr=self.sample_rate))
        down_list.append(librosa.feature.spectral_rolloff(y=audio, sr=self.sample_rate))

tr = Transformer()

In [12]:
amplitude_list = []
mfccs_list = []
rms_list = []
zcr_list = []
stft_list = []
cent_list = []
down_list = []

In [13]:
%time
from tqdm.autonotebook import tqdm

for i in tqdm(train['name']):
    tr.obrabotka(f'../new-data-without_silence/train/{i}')

/var/folders/49/9rmnvkk90fzd67qfbq034rhr0000gn/T/ipykernel_65104/2352580041.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


CPU times: user 2 μs, sys: 5 μs, total: 7 μs
Wall time: 6.91 μs


100%|██████████| 8803/8803 [07:31<00:00, 19.49it/s]


In [14]:
d = {
    'amplitude': list(map(lambda x: x.reshape(-1, 1),amplitude_list)),
    'mfccs': list(map(lambda x: x.reshape(-1, 39),mfccs_list)), 
    'rms': list(map(lambda x: x.reshape(-1, 1),rms_list)),
    'zcr': list(map(lambda x: x.reshape(-1, 1),zcr_list)),
    'stft': list(map(lambda x: x.reshape(-1, 257),stft_list)),
    'cent': list(map(lambda x: x.reshape(-1, 1),cent_list)),
    'down': list(map(lambda x: x.reshape(-1, 1),down_list)),
}
shapes = {
    'amplitude': (None, 1),
    'mfccs': (None, 39), 
    'rms': (None, 1),
    'zcr': (None, 1),
    'stft': (None, 257),
    'cent': (None, 1),
    'down': (None, 1),
}
types = {c: np.float32 for c in shapes.keys()}

In [15]:
import tensorflow as tf

def create_cnn(model_name: str, input_shape: tuple):
    try:
        input = tf.keras.layers.Input(shape=input_shape)

        x = tf.keras.layers.Conv1D(8, kernel_size=3, activation='relu')(input)
        x = tf.keras.layers.Conv1D(16, 3, activation='relu')(x)
        x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
        x = tf.keras.layers.Dropout(0.25)(x)
        x = tf.keras.layers.GlobalAveragePooling1D()(x)
        
        model = tf.keras.Model(inputs=input, outputs=x)

        return model
    except:
        print(model_name)

In [16]:
models = {
    name: create_cnn(name, shape) for name, shape in shapes.items()
}

2024-08-27 22:34:35.846327: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-08-27 22:34:35.846348: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-27 22:34:35.846353: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-27 22:34:35.846365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-27 22:34:35.846376: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
inputs = {name: tf.keras.layers.Input(shape=shape, name=name) for name, shape in shapes.items()}
outputs = [model(inputs[name]) for name, model in models.items()]

y = tf.keras.layers.Concatenate(axis=1)(outputs)
y = tf.keras.layers.Dense(1, activation='sigmoid')(y)

model = tf.keras.Model(inputs=inputs, outputs=y)

In [18]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['f1_score'])

In [19]:
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ amplitude           │ (None, None, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mfccs (InputLayer)  │ (None, None, 39)  │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rms (InputLayer)    │ (None, None, 1)   │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zcr (InputLayer)    │ (None, None, 1)   │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stft (InputLayer)   │ (None, None, 257) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cent (InputLayer)   │ (None, None, 1)   │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ down (InputLayer)   │ (None, None, 1)   │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 16)        │        432 │ amplitude[0][0]   │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 16)        │      1,344 │ mfccs[0][0]       │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 16)        │        432 │ rms[0][0]         │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, 16)        │        432 │ zcr[0][0]         │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_4        │ (None, 16)        │      6,576 │ stft[0][0]        │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_5        │ (None, 16)        │        432 │ cent[0][0]        │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_6        │ (None, 16)        │        432 │ down[0][0]        │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 112)       │          0 │ functional[0][0], │
│ (Concatenate)       │                   │            │ functional_1[0][… │
│                     │                   │            │ functional_2[0][… │
│                     │                   │            │ functional_3[0][… │
│                     │                   │            │ functional_4[0][… │
│                     │                   │            │ functional_5[0][… │
│                     │                   │            │ functional_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │        113 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,193 (39.82 KB)

 Trainable params: 10,193 (39.82 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
for data in d.values():
    print(len(data))

8803
8803
8803
8803
8803
8803
8803


In [26]:
dataset = tf.data.Dataset.from_generator(
    lambda: (d, train['label']),
    output_types=(types, np.int16)
)

In [27]:
model.fit(dataset)

ValueError: as_list() is not defined on an unknown TensorShape.